In [1]:
#!python --version
#nötig ist version 3.8.17

### Bibliotheken, die ich benutze

In [2]:
import mediapipe as mp    #model erstellen
import numpy as np        #mit arrays arbeiten
import os                 #mit dem Dateisystem interagieren

### Load my model

In [3]:
from sklearn.model_selection import train_test_split         #datensatz teilen für testen und trainieren
from tensorflow.keras.utils import to_categorical          

# Liste aller Gesten abrufen
# https://www.geeksforgeeks.org/python-list-files-in-a-directory/

path = os.path.join('datensatz1')
gesten_list = os.listdir(path)

#np array ist später nötig für trainieren
gesten = np.array(gesten_list)


#liste in der dictionary umwandeln und numarieren
label_map = {}
n = 0
for label in gesten_list:
    label_map[gesten_list[n]] = n
    n += 1

#wir müssen hier anpassen mithilfe von unsere datensatz, wie viele videos wir pro geste aufgenommen haben und wie viele frame hat jede video
num_videos = 30
vid_length = 20

#mein datensatz verzeichnis
DATA_PATH = os.path.join('datensatz1')

videos = []
labels = []

#alle videos k-points in eine array zusammenfassen
for geste in gesten:
    for vid in range(num_videos):
        window = []
        for frame_num in range(vid_length):
            res = np.load(os.path.join(DATA_PATH, geste, str(vid), "{}.npy".format(frame_num)))
            window.append(res)
        videos.append(window)
        labels.append(label_map[geste])

#umwandeln in numpy array
#kp = k-points
kp = np.array(videos)

#gn = geste_name
gn = to_categorical(labels).astype(int)

#datensatz teilen in test- und trainierenteil
kp_train, kp_test, gn_train, gn_test = train_test_split(kp, gn, test_size=0.05)

In [4]:
from tensorflow.keras.models import Sequential
#Die sequentielle ist nützlicher als die funktionale, da wir eine Dateneingabe und eine Datenausgabe haben.
#Es leitet die Daten weiter und fließt in sequentieller Reihenfolge von oben nach unten, bis die Daten am Ende des Modells ankommen.
#https://www.educba.com/keras-sequential/
#https://www.youtube.com/watch?v=8uC-WT1LYnU&ab_channel=Simplilearn

from tensorflow.keras.layers import LSTM, Dense
#LSTM Layer: ermöglicht uns die Erkennung von Aktionen
#Keras Dense-Schicht ist die Schicht, die alle Neuronen enthält, die in sich selbst tief verbunden sind. Das bedeutet, dass jedes Neuron in der dichten Schicht die Eingabe von allen anderen Neuronen der vorherigen Schicht erhält. Wir können so viele dichte Schichten wie nötig hinzufügen. Sie ist eine der am häufigsten verwendeten Schichten.
#https://www.educba.com/keras-dense/
#https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense


model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20, 258)))
#64 LSTM Units     
#return sequences = True: weil die nächste Schicht dies braucht
#activation = relu: Funktion, die die Eingabe direkt ausgibt, wenn sie positiv ist, andernfalls gibt sie Null aus.

model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
#return sequences = False: weil die nächste Schicht ist Dense layer/schicht

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

#letzte Schicht
model.add(Dense(gesten.shape[0], activation='softmax'))
#Softmax ist eine Aktivierungsfunktion, die Zahlen/Logits in Wahrscheinlichkeiten umwandelt. 
#Die Ausgabe einer Softmax-Funktion ist ein Vektor (z. B. v) mit Wahrscheinlichkeiten für jedes mögliche Ergebnis. 
#Die Wahrscheinlichkeiten im Vektor v summieren sich für alle möglichen Ergebnisse oder Klassen zu eins.
#https://towardsdatascience.com/softmax-activation-function-how-it-actually-works-d292d335bd78

In [5]:
#load model/gewichte

model.load_weights("gesten1.h5")

In [6]:
#Testen, ob das Modell da ist

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 237350 (927.15 KB)
Trainable params: 23735

In [7]:
#siehe Genauigkeit der Modelle

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(kp_train)
ytrue = np.argmax(gn_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
accuracy_score(ytrue, yhat)

6/6 [==============================] - 1s 21ms/step


1.0

### Test in Real Time

In [8]:
import cv2 as cv      #zugriff auf kamera

In [9]:
#modelle erstellen

mp_holistic = mp.solutions.holistic
#holistic kombiert hände, körper und gesicht erkennung
#https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
#https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md

mp_drawing = mp.solutions.drawing_utils

#farben umwandlung + model hinzufügen
def conversion(frame, model):
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    #hier umwandeln wir frame in rgb weil mediapipe braucht rgb format
    
    frame.flags.writeable = False
    results = model.process(frame)
    #hier hinzufügen wir mediapipe model/"eine Vorhersage treffen"
    
    frame.flags.writeable = True
    frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    #hier zurückkehren wir in BGR damit cv2 es zeigen kann
    
    return frame, results

#orientierungspunkte zeigen lassen an cv
def show_kpoints(main_frame, results):
    #wir werden gesichtspunkte nicht gebrauchen bei gebärdensprache
    #mp_drawing.draw_landmarks(main_frame, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    
    #                         image       orientierungspunkte     verbindungskarte
    mp_drawing.draw_landmarks(main_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(main_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(main_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [10]:
#k-points extrahieren

#wir verwenden die "try, except"-Methode, falls nicht alle Schlüsselpunkte in der Kamera angezeigt werden, so dass es nicht zu einem Fehler kommt
def extract_keypoints(results):
    try:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    except:
        pose = np.zeros(33*4)

    try:
        linke_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    except:
        linke_hand = np.zeros(21*3)

    try:
        rechte_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    except:
        rechte_hand = np.zeros(21*3)
    
    return np.concatenate([pose, linke_hand, rechte_hand])

In [11]:
num_frames = []
satz = []
threshold = 0.6


capture = cv.VideoCapture(1)
# 1 = mein computer webcam
# 0 = mein externe webcam/handy


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while capture.isOpened():
        isTrue, frame = capture.read()

        flipped_frame = cv.flip(frame, 1)   #bild spiegeln

        main_frame, results = conversion(flipped_frame, holistic)

        #orientierungspunkte in kamera zeigen lassen
        show_kpoints(main_frame, results)
        
        #k-points speichern in variable
        kpoints = extract_keypoints(results)
        num_frames.append(kpoints)
        num_frames = num_frames[-30:]
        
        #wir führen die Vorhersage nur aus, wenn es 30 Frames sind
        if len(num_frames) == 30:
            res = model.predict(np.expand_dims(num_frames, axis = 0))[0]   #np expand dims makes sequence in required shape
            print(gesten[np.argmax(res)])
            
            if res[np.argmax(res)] > threshold: 
                if len(satz) > 0: 
                    if gesten[np.argmax(res)] != satz[-1]:
                        satz.append(gesten[np.argmax(res)])
                else:
                    satz.append(gesten[np.argmax(res)])
            
            #nur eine wort zeigen
            if len(satz) > 1: 
                satz = satz[-1:]
        
        #platz für geste "wort"
        cv.rectangle(main_frame, (0,0), (650, 50), (255, 255, 255), -1)
        cv.putText(main_frame, ' '.join(satz), (240,45), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 3, cv.LINE_AA)
        
        cv.putText(main_frame, 'Projekt von Zaza Buzaladze', (370,475), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)
            
        cv.imshow("Kamera - Gestus", main_frame)  #Anzeigen, Wiedergeben des Bildes
        
        if cv.waitKey(20) & 0xFF==ord("s"):
            break
            #mit "s"(stop), stoppt kamera    

    capture.release()
    cv.destroyAllWindows()

In [12]:
#cv fenster ausschalten (im absturz fall)
capture.release()
cv.destroyAllWindows()

# Quellen

https://www.youtube.com/watch?v=doDUihpj6ro&list=PLyx6mI41rWldgZWHzHBGBQfP5MxFQNcFf&index=5&ab_channel=NicholasRenotte
https://www.geeksforgeeks.org/python-list-files-in-a-directory/
https://www.educba.com/keras-sequential/
https://www.youtube.com/watch?v=8uC-WT1LYnU&ab_channel=Simplilearn
https://www.educba.com/keras-dense/
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
https://towardsdatascience.com/softmax-activation-function-how-it-actually-works-d292d335bd78
https://developers.google.com/mediapipe/solutions/vision/holistic_landmarker
https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md